In [5]:
import pandas as pd
import math
import glob
from collections import defaultdict
import os # these two packages are good for searching and navigating file systems
import os.path as op

pd.set_option('display.max_columns', None)

#path = '/Users/melissaherring/Google Drive/My Drive/MH_project/'
path = '/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/'

user_path = ''

# function to split classification and get rid of _
def split_classification(df):
    df[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']] = df.classification.str.split(';', expand=True)
    df['domain'] = df['domain'].str.replace('d__', '') # remove the d__ in front of all observations
    df['phyla'] = df['phyla'].str.replace('p__', '')
    df['class'] = df['class'].str.replace('c__', '')
    df['order'] = df['order'].str.replace('o__', '')
    df['family'] = df['family'].str.replace('f__', '')
    df['genus'] = df['genus'].str.replace('g__', '')
    df['species'] = df['species'].str.replace('s__', '')
    return df

In [17]:
# for loop: create meta data columns for result data frame
# if data_type = SAG use sag lookup table
# if data_type = MAG use mag lookup table

result = pd.read_csv(op.join(path, 'dramv_full_summary.csv'))

sag_cols = ['vir_id','depth', 'classification_via_GTDBTk']
sag_tax = pd.read_csv(op.join(path, 'data/sag_data/MPvsag_info_230818.csv'))[sag_cols]
sag_tax.rename(columns={'classification_via_GTDBTk': 'classification','depth':'sample_depth'}, inplace=True)
sag_tax = split_classification(sag_tax)
sag_tax['virus_name'] = sag_tax['vir_id'].str.replace('vir', 'cv1')

mag_tax = pd.read_csv(op.join(path, 'tables/vMAG_associations.csv'))
mag_cols= ['virus_name', 'sample_depth','domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']

def rename_vmag(virid, sample_name):
    new_samp = '{}-{}'.format(sample_name[:2].lower(), sample_name[2:])
    return '{}-{}'.format(new_samp, virid)

mag_tax['virus_name'] = [rename_vmag(l['virus_name'],l['sample_name']) for i, l in mag_tax.iterrows()]

'''
# above list comprehension is the same as this code:

new_list = []

for i, l in mag_tax.iterrows():
    new_list.append(rename_vmag(l['virus_name'],l['sample_name']))
    
mag_tax['virus_name'] = new_list
'''
mag_tax = mag_tax[mag_cols]

In [18]:
tax_df = pd.concat([sag_tax,mag_tax])[mag_tax.columns]
tax_df.to_csv(op.join(path, 'tables/091923_all_vir_host_taxonomy.csv'),
              index = False)

In [15]:
#tax_df[~tax_df.isna()]

,virus_name,sample_depth,domain,phyla,class,order,family,genus,species
0,cv1_AM-654-B02,80,Bacteria,Proteobacteria,Gammaproteobacteria,Arenicellales,UBA5680,UBA5680,UBA5680 sp002420425
1,cv1_AM-654-B17,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,
2,cv1_AM-654-B06,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,GCA-2704625,GCA-2704625 sp017640245
3,cv1_AM-654-C02,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,
4,cv1_AM-654-B04,80,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,BACL11,DUAL01,
...,...,...,...,...,...,...,...,...,...
127,jv-154-vMAG_44,140,Bacteria,Planctomycetota,Planctomycetia,Pirellulales,Pirellulaceae,ARS98,ARS98 sp002685655
128,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN
129,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN
130,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN


In [20]:
result[result['ID'].str.contains('vMAG')]['ID']

0      jv-132-vMAG_16
1       jv-132-vMAG_3
6      jv-119-vMAG_13
7      jv-121-vMAG_62
8      jv-121-vMAG_76
            ...      
325    jv-154-vMAG_42
326    jv-119-vMAG_36
327    jv-119-vMAG_22
329    jv-132-vMAG_33
330    jv-132-vMAG_27
Name: ID, Length: 214, dtype: object

In [25]:
result.rename(columns = {'ID':'virus_name'}).merge(tax_df, on = 'virus_name', how = 'outer').to_csv(op.join(path, 'tables/091923_dramv_and_host_tax_all_viruses.csv'), index = False) #.dropna()

In [ ]:
for index, row in df.iterrows():
    if result['data_type'] == 'vSAG':
        result.merge(sag_tax, on=['ID','virus_name'], how='left')
    else: result.merge(mag_tax, on=['ID','vir_id'], how = 'left')